In [ ]:
import os
import numpy
import matplotlib.pyplot as plt

from ase.optimize import BFGS, FIRE
from ase.filters import FrechetCellFilter
from ase.build import bulk
from ase.phonons import Phonons
from tensorpotential.calculator.foundation_models import grace_fm, GRACEModels

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Load model

In [ ]:
calc = grace_fm(GRACEModels.GRACE_1L_MP_r6)

# Tungsten

In [ ]:
at = bulk("W")

In [ ]:
at.calc = calc

In [ ]:
at.get_potential_energy()

In [ ]:
BFGS(FrechetCellFilter(at)).run()

## Phonons

Example from https://wiki.fysik.dtu.dk/ase/ase/phonons.html

In [ ]:
# Phonon calculator
N = 7
ph = Phonons(at, calc, supercell=(N, N, N), delta=0.05)
ph.run()

In [ ]:
# Read forces and assemble the dynamical matrix
ph.read(acoustic=True)
ph.clean()

In [ ]:
path = at.cell.bandpath(npoints=100)
bs = ph.get_band_structure(path)

dos = ph.get_dos(kpts=(20, 20, 20)).sample_grid(npts=100, width=1e-3)

In [ ]:
fig = plt.figure(1, figsize=(7, 4))
ax = fig.add_axes([0.12, 0.07, 0.67, 0.85])

emax = 0.025
bs.plot(ax=ax, emin=0.0, emax=emax)

dosax = fig.add_axes([0.8, 0.07, 0.17, 0.85])
dosax.fill_between(
    dos.get_weights(), dos.get_energies(), y2=0, color="grey", edgecolor="k", lw=1
)

dosax.set_ylim(0, emax)
dosax.set_yticks([])
dosax.set_xticks([])
dosax.set_xlabel("DOS")